# Import Libraries

In [13]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 320.0 kB/s eta 0:00:0000:0100:01


In [14]:
# calculations
import numpy as np
import pandas as pd

# visuals
import matplotlib.pyplot as plt

# parsing
from pypdf import PdfReader

import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import re

# Dataset

In [28]:
df = pd.read_csv('job_data.csv')[['Job Description']]
df.head()

,Job Description
0,"<div id=""jobDescriptionText"" class=""jobsearch-..."
1,"<div id=""jobDescriptionText"" class=""jobsearch-..."
2,"<div id=""jobDescriptionText"" class=""jobsearch-..."
3,"<div id=""jobDescriptionText"" class=""jobsearch-..."
4,"<div id=""jobDescriptionText"" class=""jobsearch-..."


In [26]:
df['Job Description'][0]

'<div id="jobDescriptionText" class="jobsearch-jobDescriptionText">WE ARE LOOKING FOR TOP PERFORMERS TO GROW WITH US!\n<br><br>\nDel Taco needs people with the ability to handle multiple responsibilities and deliver the quality and guest service that defines our brand. While we look for you to bring the right skills and personality for a Shift Manager position, we\'ll also provide the training and support you need to continue to grow and advance.\n<br><br>\nOur Shift Managers support our stores and General Managers in creating an amazing guest experience, and empowering their team to be at their very best. Our Shift Managers create a culture in our restaurants that is exciting, optimistic and rewarding.\n<br><br>\nDel Taco Shift Managers have significant opportunities for growth and career advancement and are charged with supporting the General Manager with the tasks of training the next generation of leaders, guiding them to be promoted and empowering them to have a successful career.

In [33]:
#function to process job listings
def clean_text(resumeText):
    resumeText = re.sub(r'<[^>]+>', '', resumeText)     # remove html tags
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    resumeText = re.sub(r'(\w)(?<![A-Z])([A-Z])(?![A-Z])', r'\1 \2', resumeText)
    resumeText = resumeText.lower() #remove capital letters
    words = resumeText.split(' ')
    words = [word for word in words if len(word)>1]
    resumeText = ' '.join(words)
    return resumeText

In [34]:
job = clean_text(df['Job Description'][0])
job

'we are looking for top performers to grow with us del taco needs people with the ability to handle multiple responsibilities and deliver the quality and guest service that defines our brand while we look for you to bring the right skills and personality for shift manager position we ll also provide the training and support you need to continue to grow and advance our shift managers support our stores and general managers in creating an amazing guest experience and empowering their team to be at their very best our shift managers create culture in our restaurants that is exciting optimistic and rewarding del taco shift managers have significant opportunities for growth and career advancement and are charged with supporting the general manager with the tasks of training the next generation of leaders guiding them to be promoted and empowering them to have su essful career essential job functions supports general manager in supervising team that delivers effective results in the areas of

# Resume Parser

In [ ]:
reader = PdfReader("resume.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
pdf_text = page.extract_text()

In [16]:
pdf_text

'Justin Carville\nData Scientist\nAbout Me\nProfessional Experience"Every day is a new adventure." This philosophy brought\nme to Japan in 2017, where I have since leveraged my\nlove of languages to make a living. My experience\nworking in marketing and business operations got me\nexcited about coding and data, so I changed gears and\nam now on a mission to become fluent in this new field.\nTechnical Skills\nPython\nScikit-Learn\nMachine\nLearning\nNLP\nDeep Learning\nLanguages\nEnglish : native\nJapanese : business\nSpanish : conversational\nEducation\nLe Wagon - Tokyo (2023)\n#1 ranked bootcamp worldwide\n9-week intensive data science\nbootcamp\nKICL - Kyoto (2017-2019)\nJapanese language school\nPassed JLPT N2\nUniversity of Rhode Island (2010-2013)\nBachelor of Arts in Spanish, Journalism\n Graduated Magna Cum LaudeContact Info\njccarville@gmail.comTokyo, Japan\nwww.linkedin.com/in/jccarville/\nhttps://github.com/just1nt1me\nLink Academy (2019-2023)\nFreelance Writer, Editor, Trans

In [24]:
#function to process resumes
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    resumeText = re.sub(r'(\w)(?<![A-Z])([A-Z])(?![A-Z])', r'\1 \2', resumeText)
    resumeText = resumeText.lower() #remove capital letters
    words = resumeText.split(' ')
    words = [word for word in words if len(word)>1]
    resumeText = ' '.join(words)
    return resumeText

In [25]:
resume = cleanResume(pdf_text)
resume

'justin carville data scientist about me professional experience every day is new adventure this philosophy brought me to japan in 2017 where have since leveraged my love of languages to make living my experience working in marketing and business operations got me excited about coding and data so changed gears and am now on mission to become fluent in this new field technical skills python scikit learn machine learning nlp deep learning languages english native japanese business spanish conversational education le wagon tokyo 2023 ranked bootcamp worldwide week intensive data science bootcamp kicl kyoto 2017 2019 japanese language school passed jlpt n2 university of rhode island 2010 2013 bachelor of arts in spanish journalism graduated magna cum laude contact info arville japan www linkedin com in arville link academy 2019 2023 freelance writer editor translator 2018 2023 vipkid esl teacher 2017 2019 we love osaka link to articles sns video content creation for you tube instagram kpi 